# Project Machine Learning - Modelling and Evaluaation
Kelompok 4 ( WIN GAMES )

Nama Kelompok :
1. Wilhelmus Medhavi
2. Alfiyanti S.
3. Mayang Indi G
4. Sumayya
5. Anggina Dwi F.
6. Credenda M.
7. Dimas Fauzi Prasetyo

**Data Preparation**

In [1]:
# import modul 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn import under_sampling, over_sampling
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# load dataset
data_train = pd.read_csv('/content/drive/MyDrive/Source Code/Data_Train_Model.csv')
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252000 entries, 0 to 251999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            252000 non-null  int64  
 1   Age_norm              252000 non-null  float64
 2   Income_norm           252000 non-null  float64
 3   Experience_norm       252000 non-null  float64
 4   House_norm            252000 non-null  float64
 5   Car_Ownership         252000 non-null  int64  
 6   State_Rank            252000 non-null  int64  
 7   Prof_Rank             252000 non-null  int64  
 8   married_own_carhouse  252000 non-null  int64  
 9   Risk_Flag             252000 non-null  int64  
dtypes: float64(4), int64(6)
memory usage: 19.2 MB


In [4]:
# Deklarasi nilai x dan y
x = data_train[['Age_norm','Income_norm','Experience_norm','House_norm','State_Rank','Prof_Rank','married_own_carhouse']]
y = data_train[['Risk_Flag']]

In [5]:
# Melihat variabel penduga yang digunakan
x.head(5)

,Age_norm,Income_norm,Experience_norm,House_norm,State_Rank,Prof_Rank,married_own_carhouse
0,0.034483,0.129487,0.15,0.75,24,10,0
1,0.948276,0.228713,0.60,0.75,24,10,0
2,0.465517,0.946853,0.00,0.50,24,10,0
3,0.206897,0.275715,0.20,0.25,24,10,0
4,0.034483,0.129487,0.15,0.75,24,10,0


In [6]:
# Handling class imbalance menggunakan SMOTE dan undersampling
# Menggunakan SMOTE pada risk_flag=1 (customer beresiko) dan kemudian melakukan undersampling pada customer yang tidak beresiko

x_over_SMOTE, y_over_SMOTE = SMOTE(sampling_strategy=0.4,random_state=42).fit_resample(x,y)
x_under, y_under = under_sampling.RandomUnderSampler(0.5,random_state=42).fit_resample(x_over_SMOTE,y_over_SMOTE)

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:591: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning,


Penggunaan SMOTE pada customer yang beresiko (risk_flag=1) dan undersampling pada customer tidak beresiko (risk_flag=0) bertujuan untuk meningkatkan rasio data, menjadi sebesar 66,66:33,33. 

Penggunaan sampling_strategy sebesar 0,4 pada SMOTE bertujuan untuk mencegah terlalu banyak data sintetis hasil ekstrapolasi yang dibuat; sedangkan pada undersampling sebesar 0,5 bertujuan untuk meminimalkan data yang dibuang.

In [7]:
# Mengecek hasil undersampling dan SMOTE
y_under.value_counts()

Risk_Flag
0            176802
1             88401
dtype: int64

In [8]:
# Melihat perbandingan class risk_flag awal
y.value_counts()

Risk_Flag
0            221004
1             30996
dtype: int64

In [9]:
# Memisahkan data train dan test
X_train, X_test, y_train, y_test = train_test_split(x_under, y_under, test_size=0.3, random_state=42)

Data kemudian dibagi menjadi data train dan data test dengan proporsi 0,3 agar mendapatkan jumlah data test yang lebih banyak.

**Model Selection**

In [10]:
# Model Logistik
logreg = LogisticRegression(random_state=42)
# Model KNearest Neighbour
knn = KNeighborsClassifier()
# Model Decision Tree
dt = DecisionTreeClassifier(random_state=42)

# Ensemble model
# Model Random Forest
clf = RandomForestClassifier(random_state=42)
# Model XGBoost
xg = XGBClassifier(random_state=42)
# Model Adaboost
ada = AdaBoostClassifier(random_state=42)

Pemilihan model dilakukan secara trial and error dengan melihat hasil metrcs peforma model. Bentuk model yang digunakan berupa model untuk klasifikasi, seperti logistik, K-Nearest Neighbour, Decission tree, Random forest, XGBosst, dan Adaboost. 

**Model Evaluation**

In [11]:
# Membuat Fungsi untuk memanggil metric evaluation
def check_scoring(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    print('Train score: ', model.score(X_train, y_train))
    print('Test score: ', model.score(X_test, y_test))
    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print('Precision :', precision_score(y_test, y_pred))
    print('Recall: ', recall_score(y_test, y_pred))
    print('AUC: ', roc_auc_score(y_test, y_pred))
    print('F1: ', f1_score(y_test, y_pred))
    print('Classification Report: ', classification_report(y_test, y_pred))
    print('Evaluation: ', confusion_matrix(y_test, y_pred))

In [12]:
check_scoring(logreg)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Train score:  0.6676452526906627
Test score:  0.6641067859881098
Accuracy:  0.6641067859881098
Precision : 0.48672566371681414
Recall:  0.0020583062011152275
AUC:  0.5004803264540777
F1:  0.004099277036595364
Classification Report:                precision    recall  f1-score   support

           0       0.66      1.00      0.80     52840
           1       0.49      0.00      0.00     26721

    accuracy                           0.66     79561
   macro avg       0.58      0.50      0.40     79561
weighted avg       0.60      0.66      0.53     79561

Evaluation:  [[52782    58]
 [26666    55]]


In [13]:
check_scoring(knn)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train score:  0.8997532885877119
Test score:  0.8789230904588932
Accuracy:  0.8789230904588932
Precision : 0.7867498993153443
Recall:  0.8772875266644212
AUC:  0.8785188579574945
F1:  0.8295557089017463
Classification Report:                precision    recall  f1-score   support

           0       0.93      0.88      0.91     52840
           1       0.79      0.88      0.83     26721

    accuracy                           0.88     79561
   macro avg       0.86      0.88      0.87     79561
weighted avg       0.88      0.88      0.88     79561

Evaluation:  [[46486  6354]
 [ 3279 23442]]


In [14]:
check_scoring(dt)

Train score:  0.9439674211654691
Test score:  0.886514749688918
Accuracy:  0.886514749688918
Precision : 0.7866865439460721
Recall:  0.908424085924928
AUC:  0.8919296811153784
F1:  0.843183910241937
Classification Report:                precision    recall  f1-score   support

           0       0.95      0.88      0.91     52840
           1       0.79      0.91      0.84     26721

    accuracy                           0.89     79561
   macro avg       0.87      0.89      0.88     79561
weighted avg       0.89      0.89      0.89     79561

Evaluation:  [[46258  6582]
 [ 2447 24274]]


In [15]:
check_scoring(clf)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Train score:  0.9439674211654691
Test score:  0.90774374379407
Accuracy:  0.90774374379407
Precision : 0.8356017316017316
Recall:  0.9029602185546948
AUC:  0.9065614870214805
F1:  0.8679761133894526
Classification Report:                precision    recall  f1-score   support

           0       0.95      0.91      0.93     52840
           1       0.84      0.90      0.87     26721

    accuracy                           0.91     79561
   macro avg       0.89      0.91      0.90     79561
weighted avg       0.91      0.91      0.91     79561

Evaluation:  [[48093  4747]
 [ 2593 24128]]


In [16]:
check_scoring(xg)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train score:  0.7055838657200418
Test score:  0.7030454619725744
Accuracy:  0.7030454619725744
Precision : 0.9283144201494603
Recall:  0.1255192545189177
AUC:  0.5603088324070743
F1:  0.22113799696709963
Classification Report:                precision    recall  f1-score   support

           0       0.69      1.00      0.82     52840
           1       0.93      0.13      0.22     26721

    accuracy                           0.70     79561
   macro avg       0.81      0.56      0.52     79561
weighted avg       0.77      0.70      0.62     79561

Evaluation:  [[52581   259]
 [23367  3354]]


In [17]:
check_scoring(ada)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train score:  0.6801262645306558
Test score:  0.6771282412237152
Accuracy:  0.6771282412237152
Precision : 0.9013209013209014
Recall:  0.04341154896897571
AUC:  0.5205040333792645
F1:  0.08283347614967153
Classification Report:                precision    recall  f1-score   support

           0       0.67      1.00      0.80     52840
           1       0.90      0.04      0.08     26721

    accuracy                           0.68     79561
   macro avg       0.79      0.52      0.44     79561
weighted avg       0.75      0.68      0.56     79561

Evaluation:  [[52713   127]
 [25561  1160]]


Metrik evaluasi yang digunakan pada model tersebut difokuskan pada bagian accuracy dan recall, dikarenakan tujuan dari permodelan machine learning ini adalah untuk meminimalkan cost ( mengurangi jumlah customer yang berpotensi gagal bayar) sehingga nilai recall tepat untuk digunakan (mencegah nilai false negative besar).

Berdasarkan hasil tersebut dapat diketahui bahwa model random forest memiliki nilai accuracy dan nilai recall tertinggi, sehingga dipilih menjadi model terbaik yang akan dilakukan hyperparameter tunning.

**Best Model Hyperparameter Tunning**

In [ ]:
# list dari hyperparameter

# mengumpulkan semua hyperparameter pada dictionary
hyperparameters = dict(
                    n_estimators = [int(x) for x in np.linspace(100, 1000, num=10)],
                    criterion = ['gini', 'entropy'],
                    max_features = ['auto', 'sqrt', 'log2']
                    )

# Fit model
model = RandomForestClassifier(random_state=42) 
tune_clf = RandomizedSearchCV(model, hyperparameters, cv=5, scoring='recall')

In [20]:
# Melihat hasil tunning model

check_scoring(tune_clf)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 

KeyboardInterrupt: ignored

In [ ]:
y_pred_train = tune_clf.predict(X_train)
y_pred = tune_clf.predict(X_test)
print('acc score: ', accuracy_score(y_train, y_pred_train))
print('acc score: ', accuracy_score(y_test, y_pred))
print('recall score: ', recall_score(y_train, y_pred_train))
print('recall score: ', recall_score(y_test, y_pred))

Berdasarkan hasil hyperparameter tuning tersebut, nilai accuracy dan nilai recall terjadi peningkatan, sehingga hyperparameter tersebut dapat digunakan untuk meningkatkan performa model.

**Feature Importance**

In [ ]:
# Membuat fungsi untuk menampilkan feature importance dan best parameters
def show_feature_importance(model):
    feat_importances = pd.Series(model.feature_importances_, index=x.columns)
    ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(12, 8))
    ax.invert_yaxis()

    plt.xlabel('score')
    plt.ylabel('feature')
    plt.title('feature importance score')

def show_best_hyperparameter(model):
    print(model.best_estimator_.get_params())

In [ ]:
# Melihat feature importance
show_feature_importance(tune_clf.best_estimator_)

Income_norm merupakan fitur terpenting dalam model, Oleh karena itu dapat dikatakan bahwa tingkat pendapatan masing-masing customer sangat mempengaruhi resiko gagal membayar. Perlu dilakukan pencarian lebih lanjut untuk mengetahui treshold pendapatan agar memudahkan filter customer yang beresiko. Selain itu, pekerjaan dan umur customer juga perlu diperhatikan karena menempati peringkat kedua dan ketiga terpenting.

Fitur married_own_carhouse memiliki pengaruh paling rendah (fitur paling tidak penting) meskipun terdapat kecenderungan gagal bayar bagi customer yang sudah menikah, memiliki rumah, dan memiliki mobil. 


In [ ]:
# Melihat best parameter
show_best_hyperparameter(tune_clf)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 48, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 950, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}

In [21]:
# Membuang variabel married_own_carhouse
X_train.drop(['married_own_carhouse'], axis=1)
X_test.drop(['married_own_carhouse'], axis=1)

,Age_norm,Income_norm,Experience_norm,House_norm,State_Rank,Prof_Rank
235824,0.103448,0.447132,0.75,0.75,17,23
180814,0.793103,0.321738,0.00,0.75,26,38
230679,0.913793,0.347166,0.25,0.00,26,3
227872,0.724138,0.246998,0.75,0.50,13,8
2532,0.051724,0.478137,0.55,0.25,13,35
...,...,...,...,...,...,...
137826,0.034483,0.208872,0.55,0.50,2,14
58531,0.275862,0.264548,0.10,0.25,9,1
13995,0.034483,0.000259,0.60,0.50,10,49
246011,0.775862,0.448429,0.25,0.25,17,16


In [40]:
# Iterasi model berdasarkan best parameter dan feature selection
# hyperparameters = dict(bootstrap= True, ccp_alpha= 0.0, class_weight= None, criterion= 'gini', max_depth= 48, max_features= 'sqrt', max_leaf_nodes= None, max_samples= None, min_impurity_decrease= 0.0, min_samples_leaf= 1, min_samples_split= 2, min_weight_fraction_leaf= 0.0, n_estimators= 950, n_jobs= None, oob_score= False, random_state= 42, verbose= 0, warm_start= False)

# Fit model
tune_clf_new = RandomForestClassifier(bootstrap= True, ccp_alpha= 0.0, class_weight= None, criterion= 'gini', max_depth= 48, max_features= 'sqrt', max_leaf_nodes= None, max_samples= None, min_impurity_decrease= 0.0, min_samples_leaf= 1, min_samples_split= 2, min_weight_fraction_leaf= 0.0, n_estimators= 950, n_jobs= None, oob_score= False, random_state= 42, verbose= 0, warm_start= False) 
check_scoring(tune_clf_new)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Train score:  0.9439674211654691
Test score:  0.9088372443785272
Accuracy:  0.9088372443785272
Precision : 0.8375407448505444
Recall:  0.9038958122824745
AUC:  0.9076159606454008
F1:  0.8694540938461816
Classification Report:                precision    recall  f1-score   support

           0       0.95      0.91      0.93     52840
           1       0.84      0.90      0.87     26721

    accuracy                           0.91     79561
   macro avg       0.89      0.91      0.90     79561
weighted avg       0.91      0.91      0.91     79561

Evaluation:  [[48155  4685]
 [ 2568 24153]]
